# Resumen MongoDB

### Outline

0. Instalando
1. Datos
2. Importando Datos
3. Utilizando la consola de MongoDB
4. Consultando en MongoDB
    + Filtros
    + Proyección
5. Algunas consultas
6. Python y MongoDB
7. JavaScript y MongoDB


## Instalando

Acá están las instrucciones para [instalar y correr](https://docs.mongodb.com/manual/administration/install-community/). 

Para el caso de Windows, recomendamos seleccionar [Correr desde el terminal](https://docs.mongodb.com/manual/tutorial/install-mongodb-on-windows/#run-mongodb-community-edition-from-the-command-interpreter). 

Con mongo instalado, el primer paso es importar los datos. Las instrucciones de abajo asumen que estás en la carpeta con los JSONs. 

## Datos 

Trabajaremos en el contexto de una red social, con una colección de **usuarios**, otra colección de **mensajes** y una colección de productos. Los datos se ven así:

#### Usuarios

```
  "uid": <id del usuario>,
  "name": <nombre>, 
  "last_name": <apellido>, 
  "age": <edad>, 
  "occupation": <a qué se dedica>, 
  "follows": [<arreglo con una lista de ids de usuarios>]

```
#### Mensajes 

```
  "mid": <id del mensaje emitido>,
  "uid": <id del usuario que emite el mensaje>,
  "msj": {"title": <título del mensaje>,
          "content": <contenido del mensaje>},
  "likes": <número de likes del mensaje>
```

#### Productos
```
  "pid": <id del mensaje emitido>,
  "name": <id del usuario que emite el mensaje>,
  "tags": [<tag 1>, ..., <tag n>],
  "price": <precio del producto>
```

---

## Importando Datos 

Luego de haber **instalado mongo en tu Computador**, ingresa desde la consola a la carpeta donde tengas los archivos de los datos en formato `json`. Ingresa el comando `sudo service mongod start` en linux/Mac OS, o `"C:\Program Files\MongoDB\Server\4.2\bin\mongod.exe" --dbpath="c:\data\db"`en Windows (como describimos arriba, debes haber creado el diectorio data\db). Luego, ubicado en la carpata con los JSONs, para importar los datos de `usuarios.json` en la colección `usuarios`, ejecuta en la consola:


`mongoimport --db test --collection usuarios --drop --file usuarios.json --jsonArray`

Ojo: si estás en windows, ubicado en el lugar donde está el JSON, debes poner la ruta absoluta de mongoimport (tal como corrias el servidor), por ej.: 

`"C:\Program Files\MongoDB\Server\4.2\bin\mongoimport.exe" --db test --collection usuarios --drop --file usuarios.json --jsonArray`


Para importar los datos de `mensajes.json` en la colección `mensajes`:


`mongoimport --db test --collection mensajes --drop --file mensajes.json --jsonArray`


Para importar los datos de `productos.json` en la colección `productos`:

`mongoimport --db test --collection productos --drop --file productos.json --jsonArray`

---

## Utilizando la consola de MongoDB

### Consultando en MongoDB

Para utilizar MongoDB **debes tener un servidor corriendo**. Para iniciar un cliente de mongo en la consola, utiliza el comando `mongo` (o "C:\Program Files\MongoDB\Server\4.2\bin\mongo.exe" --dbpath="c:\data\db" en windows). Debería aparecer la consola de MongoDB, en la que puedes hacer consultas. Ahora ejecuta las siguientes instrucciones: 
```
> use test
> db.usuarios.find()
```
Con la primera línea nos conectamos a la base de datos `test`, y con la segunda deberías obtener a todos los usuarios. 

La consulta básica se realiza mediante la función `find()`. Opcionalmente puedes ejecutar `find` con dos parámetros que también son documentos JSON. El primer parámetro corresponde a filtros (selección) y el segundo a proyección. Ahora ejecuta la misma consulta anterior pero usando el comando `pretty()` para hacer que aparezcan los documentos de una forma un poco más legible:

```
> db.usuarios.find({}, {}).pretty()
```

**Pro Tip:** Usa el comando `help` en la consola para obtener comandos útiles para manejar  MongoDB en esta ;)

### Filtros 

El primer parámetro del comando `find()` son los filtros, o selección. La selección por igualdad es simple: la siguiente consulta busca el usuario con `uid` igual a 2:

`> db.usuarios.find({uid: 2}, {}).pretty()`


Ahora vamos a buscar todos los productos con precio mayor a 300:

`> db.productos.find({price: {$gt: 300}}, {}).pretty()`

Si te fijas, usamos `price: {$gt: 300}`. Esto nos exige que debemos buscar todos los elementos donde la key `price` sea mayor que 300. Tienes los siguientes comandos a disposición:


+ `$gt`: mayor que.
+ `$lt`: menor que.
+ `$gte`: mayor o igual que.
+ `$lte`: menor o igual que.


Ahora busquemos los productos con precio entre 200 y 400. Para esto vamos a utilizar la instrucción `$and`:

`> db.productos.find({$and: [{price: {$gte: 200}}, {price: {$lte: 400}}]}, {}).pretty()`

Como ves, esto se empieza a complicar. Vamos a escribir la consulta anterior en más de una línea para hacerla más legible:

```
db.productos.find(
  {
    $and: [
      {price: {$gte: 200}}, 
      {price: {$lte: 400}}
    ]
  }, {}).pretty();
```


La instrucción `$and` recibe un arreglo de selecciones. Se van a retornar los documentos que cumplan todas las condiciones. También existe un comando `\$or`. Finalmente, también tenemos acceso a desigualdades con la instrucción `$ne`:

`> db.usuarios.find({price: {$ne: 2}}, {}).pretty()`

En general las consultas no se ven tan amigables o legibles. Puedes ver la documentación de MongoDB para ver qué otros filtros se pueden usar. 


### Proyección

El segundo parámetro de `find()` corresponde a los campos a proyectar. Ahora proyectaremos solamente los nombres de los usuarios:

`> db.usuarios.find({}, {name: 1})`

También podemos excluir en la proyección, usando un 0 en vez de un 1:

El segundo parámetro de `find()` corresponde a los campos a proyectar. Ahora proyectaremos solamente los nombres de los usuarios:

`> db.productos.find({}, {tags: 0})`

En el caso anterior, estamos excluyendo el arreglo de tags de los productos.

---


## Ejemplos de consultas
Aquí hay algunos ejemplos breves de consultas que se pueden hacer en MongoDB.

##### Todos los mensajes:

`> db.mensajes.find({}, {}).pretty()`

##### El contenido de todos los mensajes:

`> db.mensajes.find({}, {"msj.content": 1})`

En el ejemplo anterior estamos accediendo a una key dentro de un JSON dentro de los documentos.

##### El contenido de todos los mensajes del usuario con id $i$:

`> db.mensajes.find({"uid": i}, {"msj.content": 1})`


##### El contenido de todos los mensajes del usuario con id $i$ o $j$:

`> db.mensajes.find({$or: [{"uid": i}, { "uid": j }]},  {"msj.content": 1})`

En este ejemplo estamos usando la instrucción `$or`.

##### Los usuarios mayores a 18 años ordenados por nombre:

`> db.usuarios.find({age: {$gt: 18}}, {}).sort({"name": 1})`

Aquí estamos haciendo uso de la función `sort()`.

##### Todos los productos cuyos tags sean exactamente el arreglo `["Tag 1", "Tag 2"]`, en ese orden:

`> db.productos.find({tags: ["Tag 1", "Tag 2"]}, {})`

##### Todos los productos que en sus tags contengan los elementos `"Tag 4"` y `"Tag 3"`:

`> db.productos.find({tags: {$all: ["Tag 4", "Tag 3"]}}, {})`

##### Todos los productos que en sus tags contengan los elementos `"Tag 4"` Y `"Tag 3"`:

`> db.productos.find({tags: {$all: ["Tag 4", "Tag 3"]}}, {})`

##### Todos los productos que en sus tags contengan el elemento `"Tag 5"`:

`> db.productos.find({tags: "Tag 5"}, {})`

##### Todos los productos que en sus tags contengan el elemento `"Tag 5"` o `"Tag 1"`:

`> db.productos.find({tags: {$in: ["Tag 5", "Tag 1"]}}, {})`

##### Todos los productos que en sus tags NO contengan el elemento`"Tag 5"` o `"Tag 1"`:

`> db.productos.find({tags: {$nin: ["Tag 5", "Tag 1"]}}, {})`


### El comando aggregate. 

Este comando es bastante más flexible (de hecho, es posible demostrar que [tiene el mismo poder que el álgebra relacional](https://drops.dagstuhl.de/opus/volltexte/2018/8607/pdf/LIPIcs-ICDT-2018-9.pdf)). Vamos a poner solo algunos ejemplos para que te familiarices, para mas información te recomendamos [ver la documentacion](https://docs.mongodb.com/manual/reference/operator/aggregation-pipeline/).


##### El promedio del precio de los productos:

```
> db.productos.aggregate(
  [
    {$group: {_id: null, total: {$avg: "$price"}}}
  ]
)
```


##### La suma del precio de todos los productos que contengan la etiqueta `"Tag 3"`:

```

> db.productos.aggregate(
  [
    {$match: {tags: "Tag 3"}}, 
    {$group: {_id: null, total: {$sum: "$price"}}}
  ]
);
```

##### La suma de todos los `likes` agrupadas por usuario:

```
> db.mensajes.aggregate(
  [
    {$group: {_id: "$uid", total: {$sum: "$likes"}}}
  ]
)
```

---

## Índices

Una cosa importante en MongoDB es la útilidad de los índices. 
En bases de datos relacionales podemos hacer muchas cosas sin específicar índices adicionales, pero eso es por que por defecto se indexan las llaves de cada relación. 

En mongoDB en cambio, es muy probable que tengamos que indexar ciertos campos. La idea sigue siendo siempre la misma: un índice nos permite acelerar búsquedas particulares dentro de un campo espefício de la base de datos. 
    
Por ejemplo, si tenemos muchos usuarios y queremos poder encontrar rápidamente a un usuario en particular por su `uid`, necesitamos un índice en el campo "uid" de la colección usuarios. 
    
Para construirlo, hacemos: 
    
`db.usuarios.createIndex( { uid: 1 } `
    


## Javascript + MongoDB

Aquí hay algunos ejemplos de consultas más complejas que se pueden hacer en MongoDB con la ayuda de __Javascript__. Para correr estos ejemplos, si tu solución está en el archivo `ejemplo.js`, entonces debes correr el comando:

`mongo ejemplo.js`

Las consultas de ejemplo son:

###### Cada id de usuario junto al nombre de la gente a la que sigue:

```Javascript
var cursor = db.usuarios.find({}, {});
cursor.forEach(
  (element) => {
    try {
      var follows = element["follows"];
      print("## User ID: " + element["uid"]);
      for (var followed in follows) {
        var user_cursor = db.usuarios.find({"uid": follows[followed]},{});
        user_cursor.forEach(
          (user_element) => {
            print(user_element["name"]);
          }
        )
      }
      print("##");
    }
    catch(e) {
      print("GG", e);
    }
  }
);
```



##### Cada id de usuario, junto a su nombre y la cantidad de likes total de sus mensajes:

```Javascript
var cursor = db.usuarios.find({}, {});
cursor.forEach(
  (element) => {
    try {
      var msj_cursor = db.mensajes.find({"uid": element["uid"]}, {})
      var likes = 0;
      msj_cursor.forEach(
        (msj_element) => {
          likes += msj_element["likes"];
        }
      )
      print("User ID: " + element["uid"] +
            " - Name: " + element["name"] +
            " - Likes: " + likes);
    }
    catch(e) {
      print("GG", e);
    }
  }
);
```

## Python + MongoDB = Pymongo

Para utilizar MongoDB con python deberás descargar `pymongo` y utilizar `MongoClient` para correr el servidor. 

```Python
from pymongo import MongoClient

MONGODATABASE = "test"
MONGOSERVER = "localhost"
MONGOPORT = 27017
client = MongoClient(MONGOSERVER, MONGOPORT)
mongodb = client[MONGODATABASE]

usuarios = mongodb.usuarios
mensajes = mondodb.mensajes
productos = mongodb.productos

```

Intenta el siguiente comando para mostrar todos los mensajes de la base de datos:

```Python
qfilter = mensajes.find({}, {'_id': 0})
for mensaje in qfilter:
    print(mensaje)
```
**Ojo:** Esto funciona porque en la base de datos `test` se cargaron previamente los 3 archivos indicados. 


---

Si bien los procedimientos en Javascript se pueden correr desde la terminal de mongo, vamos a realizar la primera consulta utilizando `pymongo`.

##### Cada id de usuario junto al nombre de la gente a la que sigue:

In [1]:
from pymongo import MongoClient

MONGODATABASE = "test"
MONGOSERVER = "localhost"
MONGOPORT = 27017
client = MongoClient(MONGOSERVER, MONGOPORT)
mongodb = client[MONGODATABASE]

usuarios = mongodb.usuarios
mensajes = mongodb.mensajes
productos = mongodb.productos

qusers = usuarios.find()

for user in qusers:
    print('El usuario {} sigue a: '.format(user['name']))
    for uid in user['follows']:
        ufollows = usuarios.find({'uid': uid})
        for user2 in ufollows:
            print("\t{}".format(user2['name']))
    

El usuario Florencia sigue a: 
	Adrian
	Nebil
El usuario Devil sigue a: 
	Nebil
El usuario Adrian sigue a: 
	Florencia
	Devil
El usuario Nebil sigue a: 
	Adrian
	Florencia
	Devil
El usuario Fernando sigue a: 
